# Exotic Options Pricing Models

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# import Lib
import pandas as pd
import datetime as dt
import pytz
import os
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import scipy.stats as si
import math
import networkx as nx

# import module
from datetime import datetime, timezone
from datetime import date, time
from math import trunc
from dateutil.parser import parse

## Implementing Exotic Options Pricing Models in Python

In [3]:
import numpy as np
from scipy.stats import norm

class mb_Option:
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity):
        self.S = spot_price
        self.K = strike_price
        self.r = risk_free_rate
        self.sigma = volatility
        self.T = maturity

class mb_BarrierOption(mb_Option):
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity, barrier_level):
        super().__init__(spot_price, strike_price, risk_free_rate, volatility, maturity)
        self.B = barrier_level

    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(self.T):
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            payoff += max(0, max(price_path) - self.K) if max(price_path) > self.B else 0
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_AsianOption(mb_Option):
    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(self.T):
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            average_price = np.mean(price_path)
            payoff += max(0, average_price - self.K)
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_BinaryOption(mb_Option):
    def mb_price_option(self):
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)
        return np.exp(-self.r * self.T) * norm.cdf(d2)

class mb_LookbackOption(mb_Option):
    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(self.T):
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            payoff += max(0, max(price_path) - self.K)
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_RainbowOption(mb_Option):
    def __init__(self, spot_price1, spot_price2, strike_price, risk_free_rate, volatility1, volatility2, maturity, correlation):
        super().__init__(spot_price1, strike_price, risk_free_rate, volatility1, maturity)
        self.S2 = spot_price2
        self.sigma2 = volatility2
        self.rho = correlation

    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path1 = [self.S]
            price_path2 = [self.S2]
            for _ in range(self.T):
                Z1 = np.random.standard_normal()
                Z2 = self.rho * Z1 + np.sqrt(1 - self.rho**2) * np.random.standard_normal()
                price_path1.append(price_path1[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * Z1))
                price_path2.append(price_path2[-1] * np.exp((self.r - 0.5 * self.sigma2**2) + self.sigma2 * Z2))
            payoff += max(0, max(price_path1[-1], price_path2[-1]) - self.K)
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_ChooserOption(mb_Option):
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity, choosing_time):
        super().__init__(spot_price, strike_price, risk_free_rate, volatility, maturity)
        self.t = choosing_time

    def mb_price_option(self):
        d1 = (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.t) / (self.sigma * np.sqrt(self.t))
        d2 = d1 - self.sigma * np.sqrt(self.t)
        return self.S * norm.cdf(d1) - np.exp(-self.r * self.t) * self.K * norm.cdf(d2)


#### Illustrative examples

In [6]:
# Define parameters
S = 150  # Apple's current stock price
K = 155  # Strike price
r = 0.01  # Risk-free rate
sigma = 0.2  # Volatility
T = 1  # Time to maturity (in years)

# Barrier Option
B = 160  # Barrier level
barrier_option = mb_BarrierOption(S, K, r, sigma, T, B)
print(f"Barrier Option Price: ${barrier_option.mb_price_option():.2f}")

# Asian Option
asian_option = mb_AsianOption(S, K, r, sigma, T)
print(f"Asian Option Price: ${asian_option.mb_price_option():.2f}")

# Binary Option
binary_option = mb_BinaryOption(S, K, r, sigma, T)
print(f"Binary Option Price: ${binary_option.mb_price_option():.2f}")

# Lookback Option
lookback_option = mb_LookbackOption(S, K, r, sigma, T)
print(f"Lookback Option Price: ${lookback_option.mb_price_option():.2f}")

# Rainbow Option
S2 = 140  # Price of a second stock
sigma2 = 0.18  # Volatility of the second stock
rho = 0.5  # Correlation between the two stocks
rainbow_option = mb_RainbowOption(S, S2, K, r, sigma, sigma2, T, rho)
print(f"Rainbow Option Price: ${rainbow_option.mb_price_option():.2f}")

# Chooser Option
t = 0.5  # Time at which the option type is chosen
chooser_option = mb_ChooserOption(S, K, r, sigma, T, t)
print(f"Chooser Option Price: ${chooser_option.mb_price_option():.2f}")


Barrier Option Price: $10.18
Asian Option Price: $4.33
Binary Option Price: $0.41
Lookback Option Price: $10.62
Rainbow Option Price: $12.68
Chooser Option Price: $6.63
